In [1]:
# 实现AlexNet模型
import d2lzh as d2l
from mxnet import gluon,init,nd
from mxnet.gluon import data as gdata,nn
import os
import sys

net=nn.Sequential()
# 使用较大的11 x 11窗口来捕获物体。同时使⽤步幅4来较⼤幅度减小输出高和宽。这⾥使⽤的输出通 
# 道数比LeNet中的也要大很多 
net.add(nn.Conv2D(96,kernel_size=11,strides=4,activation='relu'),#(n-k+p+s)/s
       nn.MaxPool2D(pool_size=3,strides=2),
       # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数 
       nn.Conv2D(256,kernel_size=5,padding=2,activation='relu'),
       nn.MaxPool2D(pool_size=3,strides=2),
       # 连续3个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步增大了输出通道数。
       # 前两个卷积层后不使用池化层来减小输入的高和宽 
       nn.Conv2D(384,kernel_size=3,padding=1,activation='relu'),
       nn.Conv2D(384,kernel_size=3,padding=1,activation='relu'),
       nn.Conv2D(256,kernel_size=3,padding=1,activation='relu'),
       nn.MaxPool2D(pool_size=3,strides=2), 
       # 这里全连接层的输出个数比LeNet中的大数倍。使用丢弃层来缓解过拟合 
       nn.Dense(4096,activation='relu'),nn.Dropout(0.5),
       nn.Dense(4096,activation='relu'),nn.Dropout(0.5), 
       # 输出层。由于这里使用Fashion-MNIST，所以类别数为10，而非论⽂中的1000 
       nn.Dense(10)
       )

In [2]:
# 构造⼀个⾼和宽均为224的单通道数据样本来观察每⼀层的输出形状。 
X=nd.random.uniform(shape=(1,1,224,224))
net.initialize()
for layer in net:
    X=layer(X)
    print(layer.name,'output shape:\t',X.shape)

conv0 output shape:	 (1, 96, 54, 54)
pool0 output shape:	 (1, 96, 26, 26)
conv1 output shape:	 (1, 256, 26, 26)
pool1 output shape:	 (1, 256, 12, 12)
conv2 output shape:	 (1, 384, 12, 12)
conv3 output shape:	 (1, 384, 12, 12)
conv4 output shape:	 (1, 256, 12, 12)
pool2 output shape:	 (1, 256, 5, 5)
dense0 output shape:	 (1, 4096)
dropout0 output shape:	 (1, 4096)
dense1 output shape:	 (1, 4096)
dropout1 output shape:	 (1, 4096)
dense2 output shape:	 (1, 10)


In [3]:
# 5.6.3 读取数据
# 本函数已保存在d2lzh包中方便以后使用
# os.path.join('~','.mxnet','datasets','fashion-mnist')在逗号之间添加路径分隔符\\
def load_data_fashion_mnist(batch_size,resize=None,root=os.path.join('~','.mxnet','datasets','fashion-mnist')):
    root=os.path.expanduser(root) # 将'~'展开为用户路径
    transformer=[]
    if resize:# Resize实例将图像设置为给定大小。
        transformer+=[gdata.vision.transforms.Resize(resize)]
    transformer+=[gdata.vision.transforms.ToTensor()]
    # 用Compose实例来将这两个(Resize和Totensor)变换串联
    transformer=gdata.vision.transforms.Compose(transformer)
    # 第⼀次调⽤时会自动从⽹上获取数据保存在root路径，若不指定root，则保存在默认路径。
    mnist_train=gdata.vision.FashionMNIST(root=root,train=True)# 训练集
    mnist_test=gdata.vision.FashionMNIST(root=root,train=False)# 测试集
    num_workers=0 if sys.platform.startswith('win32') else 4
    # transform_first将transformer的变换应用在数据样本的第一个的元素即图像上
    # gdata.DataLoader读取batch_size大小的数据样本
    train_iter=gdata.DataLoader(mnist_train.transform_first(transformer),batch_size,shuffle=True,num_workers=num_workers)
    test_iter=gdata.DataLoader(mnist_test.transform_first(transformer),batch_size,shuffle=False,num_workers=num_workers)
    return train_iter,test_iter

batch_size=128
train_iter,test_iter=load_data_fashion_mnist(batch_size,resize=224)

In [1]:
# 5.6.4 训练 
# 与LeNet相比，使用了更小的学习率。
import mxnet as mx
lr,num_epochs,ctx=0.01,5,mx.cpu()
net.initialize(force_reinit=True,ctx=ctx,init=init.Xavier())
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
d2l.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

KeyboardInterrupt: 

In [5]:
root=os.path.join('~','.mxnet','datasets','fashion-mnist')
root

'~\\.mxnet\\datasets\\fashion-mnist'

In [ ]:
d2l.gloss??
d2l.load_data_fashion_mnist??

In [7]:
os.path.expanduser??
os.path??

gdata.vision.transforms.ToTensor??
gdata.vision.transforms.Resize??
gdata.vision.transforms.Compose??



gdata.vision.FashionMNIST.transform??
gdata.DataLoader??
gdata.vision.FashionMNIST??
d2l.train_ch5??
gdata.vision.FashionMNIST??